In [10]:
import os
import webbrowser
import matplotlib.pyplot as plt
import folium
import pandas as pd
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import HTML, display
from project47.flp_data_vis import *
from project47.data import *
from project47.routing import *
from project47.customer import Customer
from project47.flp_func import *


Status:  Optimal
Establish facility at site  0
Establish facility at site  2
The cost of travel =  101583.39454354667


In [11]:
k = 2

coord = list(map(list, zip(lat, lon)))
fac_coord = list(map(list, zip(fac_lat, fac_lon)))

sol_fac_coord = list(map(list, zip( sol_fac_lat, sol_fac_lon)))
sol_fac_coord = [[-43.5206429, 172.6566054], [-43.5262993, 172.5707267]]
sol_fac_addr = ['19-23 Memorial Ave., Fendalton','288 Stanmore Rd, Richmond']
print(sol_fac_coord)

[[-43.5206429, 172.6566054], [-43.5262993, 172.5707267]]


In [12]:
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
# for i in range(len(sol_fac_coord )):
#     popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
#     folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
for i in range(len(lat)):
    folium.CircleMarker([lat[i],lon[i]], radius=100*weight[i], fill_opacity=0.7, color = "black").add_to(m)
for i in range(len(fac_lat)):
    folium.CircleMarker([fac_lat[i],fac_lon[i]], radius=0.5, fill_opacity=0.3, fill=True).add_to(m) 
    
m



In [4]:
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
for i in range(len(sol_fac_coord )):
    popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
    folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
for i in range(len(lat)):
    folium.CircleMarker([lat[i],lon[i]], radius=100*weight[i], fill_opacity=0.7, color = "black").add_to(m)
for i in range(len(assigned_lat)):
    for j in range(len(assigned_lat[i])):
        folium.PolyLine([(assigned_lat[i][j],assigned_lon[i][j]), (sol_fac_coord[i])], color="red", weight=0.5, opacity=1).add_to(m)

m


In [20]:

# direct to data folder
from numpy.random import Generator, PCG64
import random

cd = "/Users/karen.w/Desktop/project47_last_mile_logistics/data"
sample_data_csv = os.path.join(cd, "Toll_CHC_November_Sample_Data.csv")
# CHC_data_csv = os.path.join(cd, "christchurch_street.csv")
CHC_data_csv = "/Users/karen.w/Desktop/project47_last_mile_logistics/data/christchurch_street.csv"
sample_data_csv = "/Users/karen.w/Desktop/project47_last_mile_logistics/data/Toll_CHC_November_Sample_Data.csv"
sample_df, sample_sub_dict, CHC_df, CHC_df_grouped, CHC_sub_dict = read_data(
    sample_data_csv, CHC_data_csv)
seed = 123456789
rg = Generator(PCG64(seed))
latitude, longitude = get_sample(
    60,
    rg,
    cd,
    sample_df,
    sample_sub_dict,
    CHC_df_grouped,
    CHC_sub_dict,
    save=False,)
# cd = "/Users/karen.w/Desktop/project47_last_mile_logistics/data"

# get a random list of 20 numbers set at futile count threshld
rd = random.sample(range(0, len(latitude) - 1), 20)
futile_lat = []
futile_lon = []
for ind in rd:
    futile_lat.append(latitude[ind])
    futile_lon.append(longitude[ind])

assigned_lat = [[] for i in range(len(sol_fac_lat))]
assigned_lon = [[] for i in range(len(sol_fac_lat))]
# calculate the distance
for i in range(len(futile_lat)):
    lat_all = []
    lon_all = []
    # get the dist from the cusomter's house to the collection points
    lat_all = sol_fac_lat[:]
    lon_all = sol_fac_lon[:]
    lat_all.insert(0, futile_lat[i])
    lon_all.insert(0, futile_lon[i])
    coord_filename = None
    dist, tm = osrm_get_dist(
        cd,
        coord_filename,
        lat_all,
        lon_all,
        source=[0],
        save=False,
        host="localhost:5000",)
    # choose the closest collection point
    min_value = min(dist[0])
    # allow the package to be assigned to the closest collection point if the dist is within the threshold
    if min_value < 6000:
        min_ind = dist[0].index(min_value)
        # assign the package to its closest collection point
        assigned_lat[min_ind].append(futile_lat[i]) 
        assigned_lon[min_ind].append(futile_lon[i]) 
       
coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)

for i in range(len(sol_fac_coord )):
    popup = folium.Popup(sol_fac_addr[i], max_width=150, max_height = 40, color = "red")
    folium.Marker(sol_fac_coord[i], popup="Optimal CP").add_to(m)
    folium.Circle(sol_fac_coord[i],radius=6000, weight = 1.2).add_to(m)
for i in range(len(latitude)):
    folium.CircleMarker([latitude[i], longitude[i]], radius=2.5, fill_opacity=1, fill=True , color = "black").add_to(m)
for i in range(len(futile_lat)):
    folium.CircleMarker([futile_lat[i], futile_lon[i]], radius=2.5, fill_opacity=1, fill=True, color = "red" ).add_to(m)
# for i in range(len(assigned_lat)):
#     for j in range(len(assigned_lat[i])):
#         folium.PolyLine([(assigned_lat[i][j],assigned_lon[i][j]), (sol_fac_coord[i])], color="red", weight=0.7, opacity=1).add_to(m)

m

In [19]:
# from folium.features import DivIcon

depo = [-43.5111688, 172.7319266]
# depo = [-43.5320, 172.6397]

alat = [-43.5111688, -43.52704282, -43.52451417, -43.51341075, -43.53609232, -43.50105713]
alon = [172.7319266, 172.6861508, 172.6314524, 172.57364980000003, 172.5287917, 172.7048862]
lat5, lon5 = get_sample(
    5,
    rg,
    cd,
    sample_df,
    sample_sub_dict,
    CHC_df_grouped,
    CHC_sub_dict,
    save=False,)
lat5.insert(0, depo[0])
lon5.insert(0, depo[1])

distances, tm = osrm_get_dist(
        cd,
        coord_filename,
        lat5,
        lon5,
        source=[],
        save=False,
        host="localhost:5000",)
locs = 6
depo = 0
# distances = np.array([[0, 2, 2], [2, 0, 4], [2, 4, 0]])
r = ORToolsRouting(locs, 1, depo)
dim, ind = r.add_dimension(distances, 0, 10, True, "distance")
r.routing.SetArcCostEvaluatorOfAllVehicles(ind)
s = r.solve()
route = s.routes[0]
route
r_lat = []
r_lon = []
for i in range(len(route)):
    r_lat.append(lat5[route[i]])
    r_lon.append(lon5[route[i]])
r_coord = list(map(list, zip(r_lat, r_lon)))

# coord_st = [-43.5320, 172.6397]
m = folium.Map(location = coord_st, zoom_start=11)
popup = folium.Popup([depo], max_width=150, max_height = 40, color = "red")
folium.Marker([-43.5112, 172.7320], popup="Depot").add_to(m)

for i in range(len(lat5)):
    folium.CircleMarker([lat5[i], lon5[i]], radius=2.5, fill_opacity=1, fill=True , color = "black").add_child(folium.Popup(str(i))).add_to(m)

folium.PolyLine(r_coord, color="red", weight=2, opacity=1).add_to(m)
m



Solved: 1 (ROUTING_SUCCESS: Problem solved successfully.)
Objective: 0
